### Обычный импорт нужных библиотек и создание 2х функций на будущее

Папка создана для агрегации и приведения к единому читаемому виду разнородных двумерных excel-таблиц, анализа полученной информации и обобщающих справок. Сбор такого объема данных в неудобной форме приводил к постоянным ошибкам подсчета и некорректному анализу. Код решил эту проблему.

2022 и 2023 года - пробные. В них сохранены начальные задумки автора.

В 2023 году код уже прекрасно работал, но **вариант 2024 года самый приятный**, так как:

1. добавлены регулярные выражения (удаляют неявные дубликаты, часто появляющиеся из-за ошибок человеческого фактора при заполнении первоначальных excel-файлов);
2. добавлены новые справки по бюджетополучателям, стоящим на особом контроле;
3. Формируются только утвержденные справки с корректными названиями в excel-файлы.

В будущем планируется:

1. Усовершенствовать форму входящих первоначальных данных;
2. Добавить количество закупаемой продукции;
3. Как максимум - создать БД (PostgreSQL), хранящую в себе данные всех собранных годов.

In [1]:
import pandas as pd
from numpy import nan
import numpy as np
from os import listdir
from os.path import isfile, join
from itertools import chain
import re

from tqdm.notebook import tqdm
tqdm.pandas()

def cleaner(tabl):
    tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
    tabl['Наименование закупки'] = tabl['Наименование закупки'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].astype('object')
    tabl = tabl.drop(index=[0,1])
    return tabl

def get_tovd_stats(tovds):
    def wrapper(row):
        tovd_stats = []
        for tovd in tovds:
            if row[tovd] != nan:
                tovd_stats.append([tovd, row["КБК"], row["Задание ГОЗ"], row["Наименование закупки"], row[tovd], row["Номер письма"]])
        return tovd_stats
    return wrapper

# очистка
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^а-я]', ' ', text)   
    text = ' '.join(text.split())
    return text

# очистка
def clear_goz(text):
    text = text.lower()
    text = re.sub(r'[^а-я0-9()]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [2]:
lbo_vr_211 = 9315.2793
oumts = ['фку всоумтс мвд россии', 'фку двоумтс мвд россии',
         'фку поумтс мвд россии', 'фку сзоумтс мвд россии',
         'фку скоумтс мвд россии', 'фку соумтс мвд россии',
         'фку уоумтс мвд россии', 'фку цоумтс мвд россии']

kbk211 = ['0302 0830292019 211', '0313 0830292019 211', '0901 0830292019 211',
          '0902 0830292019 211', '0905 0830292019 211', '0706 0830292019 211',
          '0302 0830292501 211']

In [3]:
# Переменная для обновления списка (чтобы не задвоилось/троилось)
all_files = []
path = 'Original'
all_name_files = [f for f in listdir(path) if isfile(join(path, f)) and (f.endswith(".xls") or f.endswith(".xlsx"))]

for name in all_name_files:
    source = pd.read_excel(path + "/" + name, header=1)
    del source[source.columns[-1]]
    source = cleaner(source)
    source ['Номер письма'] = name
    all_files.append(source)

C:\Users\User\AppData\Local\Temp\ipykernel_11656\857129232.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
C:\Users\User\AppData\Local\Temp\ipykernel_11656\857129232.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tabl['Наименование закупки'] = tabl['Наименование закупки'].fillna(method='ffill')
C:\Users\User\AppData\Local\Temp\ipykernel_11656\857129232.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
C:\Users\User\AppData\Local\Temp\ipykernel_11656\857129232.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
 

In [4]:
new_all_files=[]

for file in all_files:
    t=file
    tovd=t.columns[3:-2]
    t=t.apply(get_tovd_stats(tovd), axis=1)
    t = list(chain.from_iterable(t))
    new_all_files.append(pd.DataFrame(np.array(t),
                                      columns=['Бюджетополучатель', 'КБК', 
                                               'Задание ГОЗ', 'Наименование закупки', 
                                               'Сумма (руб)', '№ письма']))

In [5]:
new_all_files[0]['Сумма (руб)'] = pd.to_numeric(new_all_files[0]['Сумма (руб)'], errors='coerce')
print(new_all_files[0]['Сумма (руб)'].sum())

10687706.709999979


In [6]:
cat = pd.concat(new_all_files)
cat['Сумма (руб)'] = pd.to_numeric(cat['Сумма (руб)'], errors = 'coerce')
cat = cat.dropna(subset = 'Сумма (руб)')
cat = cat.reset_index(drop=True)
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Бюджетополучатель     1799 non-null   object 
 1   КБК                   1799 non-null   object 
 2   Задание ГОЗ           1799 non-null   object 
 3   Наименование закупки  1799 non-null   object 
 4   Сумма (руб)           1799 non-null   float64
 5   № письма              1799 non-null   object 
dtypes: float64(1), object(5)
memory usage: 84.5+ KB


In [7]:
cat['Бюджетополучатель'].unique()

array(['ФКУ "СКОУМТС МВД России"', 'ФКУ "ЦОУМТС МВД России"',
       'ФКУ "ГЦСиЗИ МВД России"', 'УМВД России по Магаданской области',
       'ДТ МВД России', 'ФКУ "СЗОУМТС МВД России"',
       'ФКУ "ПОУМТС МВД России"', 'ФКУ "СОУМТС МВД России"',
       'ФКУ "ВСОУМТС МВД России"', 'ГУ МВД России по Воронежской области',
       'ЦСН БДД МВД России', 'ФКУ "УОУМТС МВД России"',
       'ФКУ "ДВОУМТС МВД России"', 'УМВД России по Приморскому краю',
       'УМВД по Еврейской АО', 'ГУ МВД России по Саратовской области',
       'УТ МВД России по ДФО', 'УМВД России по Томской области',
       'УМВД России по Курской области',
       'УМВД России по Хабаровскому краю', 'УТ МВД России по УрФО.1',
       'УТ МВД России по ПФО', 'УМВД России по Ульяновской области',
       'УМВД России по Липецкой области',
       'МВД по Республике Саха (Якутия) ', 'ФКУ "ЦСР МВД России" ',
       'УМВД России по Калининградской области',
       'ФКУ "ГЦСиЗИ МВД России" ', 'МВД по Республике Марий Эл',
       'УМВД

In [8]:
check = clear_text(cat.loc[8, "Бюджетополучатель"])
check

'фку скоумтс мвд россии'

In [9]:
cat.loc[8, "Бюджетополучатель"]

'ФКУ "СКОУМТС МВД России"'

In [10]:
cat['Бюджетополучатель'] = cat['Бюджетополучатель'].progress_apply(clear_text)

  0%|          | 0/1799 [00:00<?, ?it/s]

In [11]:
cat['Задание ГОЗ'] = cat['Задание ГОЗ'].progress_apply(clear_goz)

  0%|          | 0/1799 [00:00<?, ?it/s]

In [12]:
cat['Бюджетополучатель'].unique()

array(['фку скоумтс мвд россии', 'фку цоумтс мвд россии',
       'фку гцсизи мвд россии', 'умвд россии по магаданской области',
       'дт мвд россии', 'фку сзоумтс мвд россии', 'фку поумтс мвд россии',
       'фку соумтс мвд россии', 'фку всоумтс мвд россии',
       'гу мвд россии по воронежской области', 'цсн бдд мвд россии',
       'фку уоумтс мвд россии', 'фку двоумтс мвд россии',
       'умвд россии по приморскому краю', 'умвд по еврейской ао',
       'гу мвд россии по саратовской области', 'ут мвд россии по дфо',
       'умвд россии по томской области', 'умвд россии по курской области',
       'умвд россии по хабаровскому краю', 'ут мвд россии по урфо',
       'ут мвд россии по пфо', 'умвд россии по ульяновской области',
       'умвд россии по липецкой области', 'мвд по республике саха якутия',
       'фку цср мвд россии', 'умвд россии по калининградской области',
       'мвд по республике марий эл', 'умвд россии по тверской области',
       'умвд россии по тульской области',
   

In [13]:
cat['Задание ГОЗ'].sort_values().unique()

array(['бумагоуничтожающая машина', 'источник бесперебойного питания',
       'комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии',
       'комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии',
       'копировально множительный аппарат', 'ламинатор',
       'многофункциональное печатающее устройство',
       'обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание единой спутниковой навигационно мониторинговой системы мвд россии',
       'обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание изделия 14т1',
       'оборудование дактилоскопирования (сканер дс45)',
       'оборудование для предоставления государственных услуг согласно перечню утвержденному мвд россии',
       'оборудование телекоммуникационных инфраструктур согласно перечню утвержденного мвд россии',
       'оборудование телекоммуникацион

In [14]:
cat['Задание ГОЗ'] = cat['Задание ГОЗ'].str.replace('утвержденного', 'утвержденному')

In [15]:
cat['Задание ГОЗ'].sort_values().unique()

array(['бумагоуничтожающая машина', 'источник бесперебойного питания',
       'комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии',
       'комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии',
       'копировально множительный аппарат', 'ламинатор',
       'многофункциональное печатающее устройство',
       'обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание единой спутниковой навигационно мониторинговой системы мвд россии',
       'обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание изделия 14т1',
       'оборудование дактилоскопирования (сканер дс45)',
       'оборудование для предоставления государственных услуг согласно перечню утвержденному мвд россии',
       'оборудование телекоммуникационных инфраструктур согласно перечню утвержденному мвд россии',
       'оконечное терминальное оборудо

###  Выгружаем базу

In [16]:
with pd.ExcelWriter('.Result\\Фин по бюджетополучателям ГОЗ 2024.xlsx') as writer:
    cat.to_excel(writer)

# ВТОРАЯ ЧАСТЬ. ОСТАТКИ

In [17]:
lbo = pd.read_excel('ЛБО 24.xlsx')
lbo['Задание ГОЗ'] = lbo['Задание ГОЗ'].fillna(method='ffill')
lbo['КБК'] = lbo['КБК'].fillna(method='ffill')
lbo['КБК'] = lbo['КБК'].astype('object')
lbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Задание ГОЗ  47 non-null     object
 1   КБК          47 non-null     object
 2   ЛБО          47 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


C:\Users\User\AppData\Local\Temp\ipykernel_11656\3959918801.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lbo['Задание ГОЗ'] = lbo['Задание ГОЗ'].fillna(method='ffill')
C:\Users\User\AppData\Local\Temp\ipykernel_11656\3959918801.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lbo['КБК'] = lbo['КБК'].fillna(method='ffill')


In [18]:
lbo['Задание ГОЗ'] = lbo['Задание ГОЗ'].progress_apply(clear_goz)

  0%|          | 0/47 [00:00<?, ?it/s]

In [19]:
lbo['Задание ГОЗ'] = lbo['Задание ГОЗ'].str.replace('утвержденного', 'утвержденному')

In [20]:
lbo

,Задание ГОЗ,КБК,ЛБО
0,бумагоуничтожающая машина,0302 0830292019 211,26548400
1,источник бесперебойного питания,0302 0830292019 211,138346000
2,комплект средств глобальной навигационной спут...,0302 0830292019 211,236940500
3,комплекты средств для проведения контроля и оц...,0302 0830292019 211,139416700
4,копировально множительный аппарат,0302 0830292019 211,72120000
5,ламинатор,0302 0830292019 211,3493200
6,многофункциональное печатающее устройство,0302 0830292019 211,357893700
7,оборудование для предоставления государственны...,0302 0830292019 211,30977800
8,оборудование телекоммуникационных инфраструкту...,0302 0830292019 211,16240000
9,оконечное терминальное оборудование и оборудов...,0302 0830292019 211,160273800


Поработаем с нашими базовыми данными-распределениями.

In [21]:
for_tail = cat.copy()
del for_tail[for_tail.columns[0]]
for_tail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   КБК                   1799 non-null   object 
 1   Задание ГОЗ           1799 non-null   object 
 2   Наименование закупки  1799 non-null   object 
 3   Сумма (руб)           1799 non-null   float64
 4   № письма              1799 non-null   object 
dtypes: float64(1), object(4)
memory usage: 70.4+ KB


In [22]:
for_tail

,КБК,Задание ГОЗ,Наименование закупки,Сумма (руб),№ письма
0,0302 0830292019 211,источник бесперебойного питания,ИБП для ПЭВМ (75 штук для УМВД России по г. Се...,645000.0,9.126пп от 07.02.2024.xlsx
1,0302 0830292019 211,источник бесперебойного питания,ИБП для ПЭВМ (75 штук для УМВД России по г. Се...,-645000.0,9.126пп от 07.02.2024.xlsx
2,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 1) (3 шту...,167700.0,9.126пп от 07.02.2024.xlsx
3,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 1) (3 шту...,-167700.0,9.126пп от 07.02.2024.xlsx
4,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 3) (1 шту...,89700.0,9.126пп от 07.02.2024.xlsx
...,...,...,...,...,...
1794,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",9000000.0,ЭБ без письма (ВР 215).xlsx
1795,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx
1796,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx
1797,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx


In [23]:
#tail = pd.concat(for_tail, keys = ['Задание ГОЗ', 'Наименование закупки', 'КБК'])[lbo] ignore_index
tail = pd.concat([lbo, for_tail])
tail['Сумма (руб)'] = tail['Сумма (руб)'] * -1
tail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 1798
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Задание ГОЗ           1846 non-null   object 
 1   КБК                   1846 non-null   object 
 2   ЛБО                   47 non-null     float64
 3   Наименование закупки  1799 non-null   object 
 4   Сумма (руб)           1799 non-null   float64
 5   № письма              1799 non-null   object 
dtypes: float64(2), object(4)
memory usage: 101.0+ KB


In [24]:
tail

,Задание ГОЗ,КБК,ЛБО,Наименование закупки,Сумма (руб),№ письма
0,бумагоуничтожающая машина,0302 0830292019 211,26548400.0,NaN,NaN,NaN
1,источник бесперебойного питания,0302 0830292019 211,138346000.0,NaN,NaN,NaN
2,комплект средств глобальной навигационной спут...,0302 0830292019 211,236940500.0,NaN,NaN,NaN
3,комплекты средств для проведения контроля и оц...,0302 0830292019 211,139416700.0,NaN,NaN,NaN
4,копировально множительный аппарат,0302 0830292019 211,72120000.0,NaN,NaN,NaN
...,...,...,...,...,...,...
1794,электронная техника средства связи и защиты ин...,0302 0830292019 215,NaN,"Электронная техника, средства связи и защиты и...",-9000000.0,ЭБ без письма (ВР 215).xlsx
1795,электронная техника средства связи и защиты ин...,0302 0830292019 215,NaN,"Электронная техника, средства связи и защиты и...",-0.0,ЭБ без письма (ВР 215).xlsx
1796,электронная техника средства связи и защиты ин...,0302 0830292019 215,NaN,"Электронная техника, средства связи и защиты и...",-0.0,ЭБ без письма (ВР 215).xlsx
1797,электронная техника средства связи и защиты ин...,0302 0830292019 215,NaN,"Электронная техника, средства связи и защиты и...",-0.0,ЭБ без письма (ВР 215).xlsx


In [25]:
tail = tail.pivot_table(index=['Задание ГОЗ', 'КБК'], values=['ЛБО', 'Сумма (руб)'], aggfunc='sum')

In [26]:
tail

,,ЛБО,Сумма (руб)
Задание ГОЗ,КБК,,
бумагоуничтожающая машина,0302 0830292019 211,2.654840e+07,-2.652443e+07
источник бесперебойного питания,0302 0830292019 211,1.383460e+08,-1.320134e+08
комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии,0302 0830292019 211,2.369405e+08,-2.324340e+08
комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии,0302 0830292019 211,1.394167e+08,-1.385108e+08
копировально множительный аппарат,0302 0830292019 211,7.212000e+07,-7.212000e+07
ламинатор,0302 0830292019 211,3.493200e+06,-3.413087e+06
многофункциональное печатающее устройство,0302 0830292019 211,3.578937e+08,-3.236651e+08
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание единой спутниковой навигационно мониторинговой системы мвд россии,0302 0830292019 214,3.000000e+07,-3.000000e+07
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание изделия 14т1,0302 0830292019 214,4.566249e+09,-4.533607e+09


In [27]:
tail1 = tail.copy()
tail1 = tail1.pivot_table(index=['Задание ГОЗ', 'КБК'], values=['ЛБО', 'Сумма (руб)'], aggfunc='sum')
tail1['Остатки']= tail1['ЛБО']+tail1['Сумма (руб)']

In [28]:
tail1

,,ЛБО,Сумма (руб),Остатки
Задание ГОЗ,КБК,,,
бумагоуничтожающая машина,0302 0830292019 211,2.654840e+07,-2.652443e+07,2.397214e+04
источник бесперебойного питания,0302 0830292019 211,1.383460e+08,-1.320134e+08,6.332620e+06
комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии,0302 0830292019 211,2.369405e+08,-2.324340e+08,4.506515e+06
комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии,0302 0830292019 211,1.394167e+08,-1.385108e+08,9.058618e+05
копировально множительный аппарат,0302 0830292019 211,7.212000e+07,-7.212000e+07,0.000000e+00
ламинатор,0302 0830292019 211,3.493200e+06,-3.413087e+06,8.011260e+04
многофункциональное печатающее устройство,0302 0830292019 211,3.578937e+08,-3.236651e+08,3.422858e+07
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание единой спутниковой навигационно мониторинговой системы мвд россии,0302 0830292019 214,3.000000e+07,-3.000000e+07,0.000000e+00
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание изделия 14т1,0302 0830292019 214,4.566249e+09,-4.533607e+09,3.264238e+07


In [29]:
tail2=tail1.copy()
tail2=tail2/1000
tail2 = tail2.drop('ЛБО', axis=1)
tail2 = tail2.drop('Сумма (руб)', axis=1)
tail2

,,Остатки
Задание ГОЗ,КБК,
бумагоуничтожающая машина,0302 0830292019 211,23.97214
источник бесперебойного питания,0302 0830292019 211,6332.62046
комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии,0302 0830292019 211,4506.51500
комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии,0302 0830292019 211,905.86180
копировально множительный аппарат,0302 0830292019 211,0.00000
ламинатор,0302 0830292019 211,80.11260
многофункциональное печатающее устройство,0302 0830292019 211,34228.58400
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание единой спутниковой навигационно мониторинговой системы мвд россии,0302 0830292019 214,0.00000
обеспечение функционирования поддержка работоспособности ремонт и техническое обслуживание изделия 14т1,0302 0830292019 214,32642.38000


In [30]:
tail2 = tail2.loc[tail2.index.isin(kbk211, level= 'КБК')]
tail2

,,Остатки
Задание ГОЗ,КБК,
бумагоуничтожающая машина,0302 0830292019 211,23.97214
источник бесперебойного питания,0302 0830292019 211,6332.62046
комплект средств глобальной навигационной спутниковой системы глонасс согласно перечню утвержденному мвд россии,0302 0830292019 211,4506.51500
комплекты средств для проведения контроля и оценки защищенности согласно перечню утвержденному мвд россии,0302 0830292019 211,905.86180
копировально множительный аппарат,0302 0830292019 211,0.00000
ламинатор,0302 0830292019 211,80.11260
многофункциональное печатающее устройство,0302 0830292019 211,34228.58400
оборудование дактилоскопирования (сканер дс45),0302 0830292019 211,871.00000
оборудование для предоставления государственных услуг согласно перечню утвержденному мвд россии,0302 0830292019 211,303.71005


In [31]:
with pd.ExcelWriter('.Result\\Остатки по заданиям ГОЗ в тыс руб 2024.xlsx') as writer:
    tail2.to_excel(writer)

# ТРЕТЬЯ ЧАСТЬ. Для корректировки

In [32]:
cat

,Бюджетополучатель,КБК,Задание ГОЗ,Наименование закупки,Сумма (руб),№ письма
0,фку скоумтс мвд россии,0302 0830292019 211,источник бесперебойного питания,ИБП для ПЭВМ (75 штук для УМВД России по г. Се...,645000.0,9.126пп от 07.02.2024.xlsx
1,фку цоумтс мвд россии,0302 0830292019 211,источник бесперебойного питания,ИБП для ПЭВМ (75 штук для УМВД России по г. Се...,-645000.0,9.126пп от 07.02.2024.xlsx
2,фку скоумтс мвд россии,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 1) (3 шту...,167700.0,9.126пп от 07.02.2024.xlsx
3,фку цоумтс мвд россии,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 1) (3 шту...,-167700.0,9.126пп от 07.02.2024.xlsx
4,фку скоумтс мвд россии,0302 0830292019 211,источник бесперебойного питания,Источник бесперебойного питания (тип 3) (1 шту...,89700.0,9.126пп от 07.02.2024.xlsx
...,...,...,...,...,...,...
1794,ут мвд россии по цфо,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",9000000.0,ЭБ без письма (ВР 215).xlsx
1795,мвд по донецкой народной республике,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx
1796,мвд по луганской народной республике,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx
1797,гу мвд россии по запорожской области,0302 0830292019 215,электронная техника средства связи и защиты ин...,"Электронная техника, средства связи и защиты и...",0.0,ЭБ без письма (ВР 215).xlsx


In [33]:
korr = cat.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=['Сумма (руб)'], aggfunc='sum')
korr['Сумма (руб)'] = korr['Сумма (руб)']/1000
korr

Сумма (руб)
Задание ГОЗ                                        Наименование закупки                               КБК                              
бумагоуничтожающая машина                          Бумагоуничтожающая машина (Тип 1)                  0302 0830292019 211  5.916048e+03
                                                   Бумагоуничтожающая машина (Тип 2)                  0302 0830292019 211  1.027889e+04
                                                   Бумагоуничтожающая машина (Тип 3)                  0302 0830292019 211  5.674839e+03
                                                   Бумагоуничтожающая машина (Тип 4)                  0302 0830292019 211  4.654648e+03
источник бесперебойного питания                    ИБП для ПЭВМ                                       0302 0830292019 211  8.815132e+04
...                                                                                                                                 ...
цифровая автоматическая телефонная станция         Цифровая АТС для нужд УМВД России по Белгородск... 0302 0830292019 211  3.321176e+03
шифровальные (криптографические) средства согла... ПАК (тип 1)                                        0302 0830292019 211  0.000000e+00
                                                   ПАК (тип 2)                                        0302 0830292019 211  0.000000e+00
                                                   Программно-аппаратный комплекс криптографическо... 0302 0830292019 211  3.232940e+05
электронная техника средства связи и защиты инф... Электронная техника, средства связи и защиты ин... 0302 0830292019 215  1.398060e+06

[577 rows x 1 columns]

In [34]:
with pd.ExcelWriter('.Result\\Для корректировок по наименованиям закупки.xlsx') as writer:
    korr.to_excel(writer)

# ЧЕТВЕРТАЯ ЧАСТЬ. Для справок

In [35]:
cat_dt = cat.loc[(cat['Бюджетополучатель'] == 'дт мвд россии') & (cat['КБК'] == '0302 0830292019 211')]
cat_gzs = cat.loc[(cat['Бюджетополучатель'] == 'фку гцсизи мвд россии') & (cat['КБК'] == '0302 0830292019 211')]
cat_giaz = cat.loc[(cat['Бюджетополучатель'] == 'фку гиац мвд россии') & (cat['КБК'] == '0302 0830292019 211')]

In [36]:
cat_oumts = cat.loc[(cat['Бюджетополучатель'].isin(oumts)) & (cat['КБК'] == '0302 0830292019 211')]

In [37]:
tail211 = tail2.loc[tail2.index.isin(kbk211, level= 'КБК')].sum()/1000
tail211 = tail211[0]
tail211

C:\Users\User\AppData\Local\Temp\ipykernel_11656\3518676027.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tail211 = tail211[0]


518.46608656

In [38]:
fin_dt = cat_dt['Сумма (руб)'].sum()/1000000
fin_oumts = cat_oumts['Сумма (руб)'].sum()/1000000
fin_gzs = cat_gzs['Сумма (руб)'].sum()/1000000
fin_tovd = lbo_vr_211-fin_gzs-fin_oumts-fin_dt-tail211
#fin_giaz = cat_giaz['Сумма (руб)'].sum()/1000000

In [39]:
with pd.ExcelWriter('.Result\\ЦЗ ДТ МВД России 211 2024.xlsx') as writer:
    cat_dt.to_excel(writer)
with pd.ExcelWriter('.Result\\Фин ГЦСиЗИ МВД России 211 2024.xlsx') as writer:
    cat_gzs.to_excel(writer)
#with pd.ExcelWriter('.Result\\Фин ГИАЦ МВД России 211.xlsx') as writer:
#    cat_giaz.to_excel(writer)

In [40]:
print('Финансирование ВР 211:', lbo_vr_211)
print()
print('Финансирование ДТ МВД России:', fin_dt)
print('Финансирование ОУМТС:', fin_oumts)
print('Финансирование ТОВД:', fin_tovd)
print('Финансирование ГЦСиЗИ:', fin_gzs)
print('Нераспределенный остаток:', tail211)
print()
print('Проверка:', fin_dt+fin_oumts+fin_gzs+tail211+fin_tovd)

Финансирование ВР 211: 9315.2793

Финансирование ДТ МВД России: 988.06187325
Финансирование ОУМТС: 6609.7298635199995
Финансирование ТОВД: 683.1385619600002
Финансирование ГЦСиЗИ: 515.88291471
Нераспределенный остаток: 518.46608656

Проверка: 9315.2793


In [41]:
tail_rub = tail211 * 1_000_000

In [42]:
tail_rub/1000

518466.08656